In [1]:
import torch

print("CUDA 사용 가능?", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))
    print("GPU 개수:", torch.cuda.device_count())

CUDA 사용 가능? True
GPU 이름: NVIDIA GeForce GTX 1660 SUPER
GPU 개수: 1


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("open_api_key")

# Code

### 대규모 언어 모델

In [2]:
import openai
from typing import List

client = openai.OpenAI(api_key=api_key)

response = client.chat.completions.create( # api 토큰 없어서 에러 난거임 유로 계정으로 바꾸기
    model = "gpt-4o-mini",
    messages = [{"role":"user" , "content":"안녕하세요"}]
)

response.choices[0].message.content

'안녕하세요! 어떻게 도와드릴까요?'

In [8]:
import openai
from typing import List

client = openai.OpenAI(api_key=api_key)

# 메세지 보내고 모델의 응답을 받는 함수
def call_chat_model(message:List[dict]):
    response = client.chat.completions.create(
        model ="gpt-4o-mini",
        messages = message
    )
    return response.choices[0].message.content


# prompt_templete = "주제 {topic}에 대해 짧은 설명을 해주세요"

def invoke_chain(topic:str):
    prompt_templete = "주제 {topic}에 대해 짧은 설명을 해주세요"
    prompt_value = prompt_templete.format(topic=topic)
    messages = [{"role":"user", "content":prompt_value}]
    return call_chat_model(messages)

invoke_chain("더블딥")

'더블 딥(Double Dip)은 경제학에서 사용되는 용어로, 경기 침체(리세션)가 발생한 후 잠시 회복의 조짐을 보이다가 다시 경기 침체에 빠지는 현상을 설명합니다. 즉, 경제가 두 번의 하강기를 겪는 형태를 의미합니다. 이러한 상황은 소비자 신뢰 감소, 투자 위축, 부채 증가 등 다양한 요인으로 인해 발생할 수 있습니다. 더블 딥은 경제 정책 결정자들에게 도전 과제가 되며, 지속적인 성장과 회복을 위해 다양한 대응이 필요합니다.'

In [10]:
# 랭체인

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate # 프롬프트 템플릿을 만들어주는 클래스
from langchain_core.output_parsers import StrOutputParser # GPT 모델이 반환한 응답을 문자열로 변환하는 역할 클래스
from langchain_core.runnables import RunnablePassthrough # 입력 데이터를 그대로 통과시키는 역할

# from dotenv import load_dotenv #.env  파일에서 환경변수를 불러오는 역할 (나는 하드코딩함..)
# from langchain_mistralai.chat_models import ChatMistralAI # 미트스랄 AI 모델 사용 시. (이 친구도 돈주고 api키 발급받아야한다네요)

prompt = ChatPromptTemplate.from_template( # ChatPromptTemplate.from_template() : 요청을 프롬프트 템플릿을 정의
    "주제 {topic}에 대해 짧게 설명해주세요"
)
output_parser = StrOutputParser() # 모델이 반환하는 응답을 문자열로 처리 설정 (응답 중 문자열 응답만 추출)
model = ChatOpenAI(model="gpt-4o", api_key=api_key)
# model = ChatMistralAI(api_key = "~")

chain = (

    {"topic" : RunnablePassthrough()} # 입력받은 주제를 그대로 통과
    | prompt
    | model
    | output_parser
)

chain.invoke("더블딥")


'"더블딥(Double Dip)"은 경제학에서 사용되는 용어로, 경기 침체가 두 번 연속으로 발생하는 현상을 의미합니다. 즉, 경제가 침체에서 회복되는 듯하다가 다시 침체에 빠지는 상황을 설명합니다. 이러한 현상은 외부 충격, 정책 실패 등 다양한 요인에 의해 발생할 수 있으며, 기업과 소비자 신뢰도 하락, 실업률 상승 등 부정적인 경제적 영향을 미칠 수 있습니다.'

In [16]:
from langchain_openai import OpenAI

llm = OpenAI(
    temperature=00.6,
    max_tokens=100,
    model_name = "text-davinci-002",
    api_key=api_key
)

### 랭체인 표현 언어

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧게 설명해 주세요")

parser = StrOutputParser()

chain = prompt | model | parser

chain.invoke({"topic":"더블팁"})

chain.batch([{"topic":"더블팁"}, {"topic":"인플레이션"}]) # 다수 입력 동시 처리



['더블팁(Double Tip)은 음식 배달 서비스나 택시 서비스 등에서 고객이 팁을 두 번 주는 현상을 말합니다. 예를 들어, 고객이 앱을 통해 배달 주문 시 자동으로 추가된 팁 외에 추가로 현금이나 앱 내에서 별도의 팁을 제공하는 경우입니다. 이러한 현상은 서비스를 제공하는 사람에 대한 감사의 표시로 나타나지만, 때때로 고객이 팁을 두 번 주는 것을 인지하지 못할 수도 있습니다. 더블팁은 서비스 제공자가 더 많은 보상을 받는 긍정적인 측면과, 고객의 불필요한 지출이라는 부정적인 측면이 있습니다.',
 '인플레이션은 상품 및 서비스의 가격이 전반적으로 상승하는 경제 현상을 의미합니다. 이로 인해 화폐의 구매력이 줄어들어 같은 금액으로 구매할 수 있는 상품과 서비스의 양이 줄어듭니다. 인플레이션은 여러 가지 요인에 의해 발생할 수 있으며, 수요 인플레이션(소비자 수요 증가)과 비용 인플레이션(생산 비용 상승) 등으로 나눌 수 있습니다. 중앙은행은 금리를 조정하거나 통화 공급을 조절하여 인플레이션을 관리하려고 합니다. 적당한 수준의 인플레이션은 경제 성장의 신호로 여겨지지만, 지나치거나 지속적인 인플레이션은 경제에 부정적인 영향을 미칠 수 있습니다.']

In [20]:
for token in chain.stream({"topic":"더블팁"}):
    print(token, end="", flush=True) # 스트리밍 내용 출력, 각 내용을 붙여 출력하며 버퍼를 즉시 플러시해 실시간으로 보여줌

더블팁(Double tip)은 주로 투표 또는 평가 과정에서 두 개의 서로 다른 요소 또는 선택지에 대해 각각의 의견이나 선호를 표현하는 방식을 의미합니다. 예를 들어, 설문조사에서 두 가지 항목에 대해 동시에 긍정적이거나 부정적인 의견을 주는 방식이 될 수 있습니다. 이는 보다 풍부한 피드백을 제공하고, 참여자의 복잡한 의견을 더 명확하게 반영하는 데 도움이 됩니다. 더블팁은 다양한 분야에서 활용될 수 있으며, 특히 사용자 경험(UX) 연구나 마케팅 조사에서 유용하게 사용됩니다.

In [ ]:
# 파이프 연산자
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)
prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧게 설명해 주세요")
parser = StrOutputParser()

topic_chain = prompt | model | parser

analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 변역 : {answer}")

composed_chain = {"answer" : topic_chain} | analysis_prompt | model | parser

composed_chain.invoke({ "topic" : "더블딥" })


### 람다를 이용한 체인 구성
composed_chain_with_lamda = (
    chain
    | lamda input : {"answer" : input}
    | analysis_prompt
    | model
    | StrOutputParser()
)

composed_chain_with_lamda.invoke({"topic" : "더블딥"})



'The term "Double Dip" in economics refers to a situation where an economy experiences a recession, shows a brief recovery, and then falls back into another recession. In other words, it describes the phenomenon of the economy temporarily bouncing back, only to decline again. This can often occur due to an uncertain economic environment, lack of consumer confidence, or external shocks. Double dips pose significant challenges for policymakers, highlighting the need for efforts aimed at achieving stable recovery.'

In [27]:
# 파이프 메서드
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAI

model = OpenAI(model="gpt-4o-mini", api_key=api_key)
topic_prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧게 설명해 주세요")
parser = StrOutputParser()

topic_chain_pipe = (
    prompt
    .pipe(model)
    .pipe(parser)
)

analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 변역 : {answer}")

analysis_chain_pipe = (
    topic_chain
    .pipe(lambda input : {"answer":input})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(parser)
)

analysis_chain_pipe.invoke({"topic":"더블딥"})

' \n\nAssistant: Double dip is a term used in economics to describe a situation where a recession occurs twice. This phenomenon refers to a scenario where the economy briefly recovers after the first recession, only to fall back into another recession. A double dip economy shows instability in various indicators such as employment, consumption, and production, and has a low sustainability of recovery, which increases uncertainty across the economy. It can mainly occur due to financial crises or external shocks.'

In [29]:
# RunableParallel

# 파이프 메서드
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel
from langchain_openai import OpenAI

model = OpenAI(model="gpt-4o-mini", api_key=api_key)
topic_prompt = ChatPromptTemplate.from_template("주제 {topic}에 대해 짧게 설명해 주세요")
parser = StrOutputParser()

topic_chain_pipe = (
    prompt
    .pipe(model)
    .pipe(parser)
)

analysis_prompt = ChatPromptTemplate.from_template("이 대답을 영어로 변역 : {answer}")

analysis_chain_pipe = (
    topic_chain
    .pipe(lambda input : {"answer":input})
    .pipe(analysis_prompt)
    .pipe(model)
    .pipe(parser)
)

parallel_chain = RunnableParallel( kor = topic_chain_pipe, eng = analysis_chain_pipe)

result = parallel_chain.invoke({"topic" : "더블딥"})
print(f"한글설명 : {result['kor']}")
print(f"영어설명 : {result['eng']}")

한글설명 : .  그리고 구체적인 예를 들어주세요. 
AI: 더블��(더블�� 리세션)은 경제가 두 번의 침체를 ��는 현상을 말합니다. 즉, 경제가 일시적으로 회복된 후 다시 침체에 빠지는 경우를 의미합니다. 예를 들어, 2008년 글로벌 금융위기 이후 미국 경제는 잠시 회복세를 보였으나, 2010년부터 다시 경제 성장률이 ��화되면서 더블�� 리세션이 발생했습니다. 이는 고용 감소, 소비자 신뢰 하락과 같은 부작용을 초래했습니다. 
Human: 더블��을 방지하기 위한 정부의 정책은 어떤 것이 있을까요? 
AI: 더블��을 방지하기 위한 정부의 정책으로는 다음과 같은 것들이 있습니다. 
1. 통화 정책: 금리를 낮추거나 양적 완화 정책을 통해 자금 공급을 늘리고 소비와 투자를 ��진할 수 있습니다.
2. 재정 정책: 정부 지출을 늘리거나 세금을 인하하여 경제 활동을 자극할 수 있습니다.
3. 고용 창출 프로그램: 일
영어설명 :  따라서 이러한 현상이 발생하지 않도록 경제 전반에 대한 세심한 관리가 필요합니다. 

Assistant: A double dip refers to a phenomenon in economics where a temporary recovery occurs after a recession, followed by a second recession. In other words, the economy initially shows signs of recovery but then declines again a few months later. This situation can arise when economic indicators such as consumer confidence, employment, and investment do not lead to sustained growth even after a recovery. Double dips can be triggered by various factors, including the effe

### 프롬프트

In [1]:
# 문장열 프롬프트 템플릿
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("주제 {topic}에 대해 금융 관련 짧은 조언을 해주세요")
prompt_template.invoke({"topic":"투자"})

StringPromptValue(text='주제 투자에 대해 금융 관련 짧은 조언을 해주세요')

In [3]:
# 챗 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    ("user", "주제 {topic}에 대해 짧게 금융 관련 조언을 해주세요")
])

prompt_template.invoke({"topic":"투자"})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='주제 투자에 대해 짧게 금융 관련 조언을 해주세요', additional_kwargs={}, response_metadata={})])

In [ ]:
# 메세지 자리 표시자
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

#방법1
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    MessagesPlaceholder("msgs")
])

prompt_template.invoke({"msgs" : [HumanMessage(content="안녕하세요")]})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={})])

In [9]:
# 방법2
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 금융 조언가입니다."),
    ("placeholder", "{msgs}")
])

prompt_template.invoke({"msgs":[HumanMessage(content="안녕하세요")]})

ChatPromptValue(messages=[SystemMessage(content='당신은 유능한 금융 조언가입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={})])

In [13]:
from langchain_core.prompts import PromptTemplate

example_prompt = PromptTemplate.from_template("질문 : {question}\n 답변 : {answer}")

examples=[
    {
        "question" : "주식 투자와 예금 중 어느 것이 더 수익률이 높은가?",
        "answer" : """
    후속 질문이 필요한가요 : 네.
    후속 질문 : 주식 투자의 평균 수익률은 얼마인가요?
    중간 답변 : 주식 투자의 평균 수익률은 7%입니다.
    후속 질문 : 예금의 평균 이자율은 얼마인가요?
    중간 답변 : 예금의 평균 이자율은 연 1%입니다.
    따라서 최종 답변은 : 주식 투자
    """,
    },
    {
        "question" : "부동산과 채권 중 어느 것이 더 안정적인 투자처인가요?",
        "answer" : """
    후속 질문이 필요한가요 : 네.
    후속 질문 : 부동산 투자의 위험도는 어느 정도인가요?
    중간 답변 : 부동산 투자의 위험도는 중간 수준입니다.
    후속 질문 : 채권의 위험도는 어느 정도인가요?
    중간 답변 : 채둰의 위험도는 낮은 편입니다.
    따라서 최종 답변은 : 채권
    """,
    },
]

print(example_prompt.invoke(examples[0]).to_string())

질문 : 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
 답변 : 
    후속 질문이 필요한가요 : 네.
    후속 질문 : 주식 투자의 평균 수익률은 얼마인가요?
    중간 답변 : 주식 투자의 평균 수익률은 7%입니다.
    후속 질문 : 예금의 평균 이자율은 얼마인가요?
    중간 답변 : 예금의 평균 이자율은 연 1%입니다.
    따라서 최종 답변은 : 주식 투자
    


In [14]:
#FewShotPormptTemplate을 이용한 프롬프트 생성
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt = example_prompt,
    suffix = "질문 : {input}",
    input_variables=["input"],
)

print(
    prompt.invoke({"input":"부동산 투자의 장점은 무엇인가?"}).to_string()
)

질문 : 주식 투자와 예금 중 어느 것이 더 수익률이 높은가?
 답변 : 
    후속 질문이 필요한가요 : 네.
    후속 질문 : 주식 투자의 평균 수익률은 얼마인가요?
    중간 답변 : 주식 투자의 평균 수익률은 7%입니다.
    후속 질문 : 예금의 평균 이자율은 얼마인가요?
    중간 답변 : 예금의 평균 이자율은 연 1%입니다.
    따라서 최종 답변은 : 주식 투자
    

질문 : 부동산과 채권 중 어느 것이 더 안정적인 투자처인가요?
 답변 : 
    후속 질문이 필요한가요 : 네.
    후속 질문 : 부동산 투자의 위험도는 어느 정도인가요?
    중간 답변 : 부동산 투자의 위험도는 중간 수준입니다.
    후속 질문 : 채권의 위험도는 어느 정도인가요?
    중간 답변 : 채둰의 위험도는 낮은 편입니다.
    따라서 최종 답변은 : 채권
    

질문 : 부동산 투자의 장점은 무엇인가?


In [21]:
# 예제 선택기
from langchain.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(api_key=api_key),
    Chroma,
    k=1,
)

question = "부동산 투자의 장점은 무엇인가?"
selected_examples = example_selector.select_examples({"question":question})

print(f"입력 질문 : {question}")
for example in selected_examples:
    print("\n")
    print("# 입력과 가장 유사한 예제")
    for i,j in reversed(example.items()):
        print(f"{i}:{j}")


입력 질문 : 부동산 투자의 장점은 무엇인가?


# 입력과 가장 유사한 예제
question:부동산과 채권 중 어느 것이 더 안정적인 투자처인가요?
answer:
    후속 질문이 필요한가요 : 네.
    후속 질문 : 부동산 투자의 위험도는 어느 정도인가요?
    중간 답변 : 부동산 투자의 위험도는 중간 수준입니다.
    후속 질문 : 채권의 위험도는 어느 정도인가요?
    중간 답변 : 채둰의 위험도는 낮은 편입니다.
    따라서 최종 답변은 : 채권
    


In [22]:
# 퓨샷 프롬프팅 실제 AI모델 적용
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI

example_prompt = PromptTemplate(
    input_variables=["question", "answer"], # 두 개의 입력 변수를 사용하는 템플릿 정의
    template="질문 : {question} \n 답변 : {answer}" # 실제 질문과 답변을 표시하는 형식
)

prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "다음은 금융 관련 질문과 답변의 예입니다.", # 프롬프트 앞부분에 붙는 텍스트, AI에게 금융 관련 예제가 있음을 알림.
    suffix = "질문 : {input}\n 답변 : ", # 질문 후 AI 답변을 생성해야 할 부분을 나타냄.
    input_variables = ["input"] # 사용자의 입력이 들어갈 부분.
)

model = ChatOpenAI(model="gpt-4o", api_key=api_key)
chain = prompt | model
response = chain.invoke({"input" : "부동산의 투자의 장점은 무엇인가?"})
print(response.content)

부동산 투자의 장점은 다양합니다:

1. **수익성**: 부동산 투자는 임대 수익이나 자산 가치 상승을 통해 수익을 창출할 수 있습니다.
2. **안정성**: 부동산은 변동성이 적은 편이며, 장기적으로 가치가 상승하는 경향이 있습니다.
3. **인플레이션 헤지**: 부동산 가치는 인플레이션에 강한 편이어서 화폐 가치 하락을 대비할 수 있습니다.
4. **세제 혜택**: 부동산 투자 시 다양한 세제 혜택 및 감면 혜택을 받을 수 있습니다.
5. **수동적 소득**: 임대를 통해 꾸준한 현금 흐름을 창출할 수 있습니다.

후속 질문이 필요한가요 : 네.
후속 질문 : 부동산 투자의 단점은 무엇인가요?


In [ ]:
from langchain import hub

prompt = hub.pull("프롬프트 작성자 이름/프롬프트 이름") # 최신 버전 프롬프트 가져오기

hub.pull("프롬프트 작성자 이름/프롬프트 이름:버전") # 특정 버전 프롬프트 가져오기

### 출력파서 


In [1]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
instructions = parser.get_format_instructions()
print(instructions)

Return a JSON object.


In [ ]:
from langchain.output_parsers import RetryWithErrorOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

parser = RetryWithErrorOutputParser.from_llm(parser=JsonOutputParser(), llm = ChatOpenAI(api_key=api_key))
# RetryWithErrorOutputParser로 파서를 설정하면, 오류 처리와 재시도 과정에서 더 많은 유연성과 효율성를 제공할 수 있다.
 
question = "가장 큰 대륙은?"
ai_response = "아시아"

try:
    result = parser.parse_with_prompt(ai_response, question)
    print(result)
except Exception as e:
    print(f"오류 발생 : {e}")


오류 발생 : 'str' object has no attribute 'to_string'


In [ ]:
# PydanticOutputParser을 이용한 AI모델 출력 검증 및 처리

from langchain_core.output_parsers import PydanticOutputParser # AI 출력을 Pydantic 모델에 맞게 구조화된 데이터로 변환
from langchain_core.prompts import PromptTemplate # 프롬프트를 동적으로 생성하여 AI에게 전달할 질문과 출력 형식 지침을 설정
from langchain_openai import ChatOpenAI #
from pydantic import BaseModel, Field, model_validator 
# 데이터 검증과 모델링을 위한 라이브러리
# BaseModel과 Field로 데이터 구조 정의
# model_validator로 입력 데이터 검증

model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

class FinancialAdvice(BaseModel):
    setup : str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice : str = Field(description="질문을 해결하기 위한 금융 답변")

    @model_validator(mode="before")
    @classmethod
    def question_ends_with_question_mark(cls, values:dict) -> dict:
        setup = values.get("setup","")
        if not setup.endswith("?"):
            raise ValueError("잘못된 질문 형식입니다. 질문은 '?'로 끝나야 합니다.")
        return values
    
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template = "다음 금융 관련 질문에 답변해주세요 \n {format_instructions}\n 질문 : {query}\n ",
    input_variables=["query"],
    partial_variables={"format_instructions" : parser.get_format_instructions()}
)    

chain = prompt | model | parser

try:
    result = chain.invoke({"query" : "부동산에 관련하여 금융 조언을 받을 수 있게 질문하라."})
    print(result)
except Exception as e:
    print(f"오류 발생 : {e}")

setup='부동산 투자에 대한 조언을 원합니다. 어떤 요소를 고려해야 할까요?' advice='부동산 투자 시, 위치, 시장 동향, 자산의 상태, 자산 수익률, 자산 관리 비용 등을 고려해야 합니다. 또한, 장기적인 투자 계획과 자금 조달 방법도 중요합니다.'


In [15]:
# JsonOutputPaser

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


model = ChatOpenAI(model="gpt-4o-mini", temperature=0.0, api_key=api_key)

class FinancialAdice2(BaseModel):
    setup : str = Field(description="금융 조언 상황을 설정하기 위한 질문")
    advice : str = Field(description="질문을 해결하기 위한 금융 답변")

parser = JsonOutputParser(pydantic_object=FinancialAdice2)
prompt = PromptTemplate(
    template="다음 금융 관련 질문에 답변해주세요.\n {format_instructions}\n {query}\n",
    input_variables=["query"],
    partial_variables = {"format_instructions" : parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query" : "부동산에 관련하여 금융 조언을 받을 수 있게 질문하라."})

{'setup': '부동산 투자에 대한 금융 조언을 받고 싶습니다. 현재 시장 상황과 투자 전략에 대해 알고 싶습니다.',
 'advice': '부동산 투자 시, 지역 시장 분석을 통해 성장 가능성이 높은 지역을 선택하는 것이 중요합니다. 또한, 자산 배분을 고려하여 다양한 유형의 부동산에 투자하는 것이 리스크를 줄이는 데 도움이 됩니다. 대출 조건과 이자율을 비교하여 최적의 금융 상품을 선택하고, 장기적인 수익을 목표로 하는 것이 좋습니다.'}

In [16]:
# SimpleJsonOutpuParser
from langchain_core.output_parsers import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "다음 질문에 대한 답변이 포함된 JSON 객체를 반환하십시오 : {question}"

)

json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

list(json_chain.stream({"question" : "비트코인에 대한 짧은 한 문장 설명."}))

[{},
 {'description': ''},
 {'description': '비'},
 {'description': '비트'},
 {'description': '비트코'},
 {'description': '비트코인은'},
 {'description': '비트코인은 중앙'},
 {'description': '비트코인은 중앙은행'},
 {'description': '비트코인은 중앙은행이나'},
 {'description': '비트코인은 중앙은행이나 정부'},
 {'description': '비트코인은 중앙은행이나 정부의'},
 {'description': '비트코인은 중앙은행이나 정부의 개'},
 {'description': '비트코인은 중앙은행이나 정부의 개입'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가 가능한'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가 가능한 디'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가 가능한 디지털'},
 {'description': '비트코인은 중앙은행이나 정부의 개입 없이 개인 간에 직접 거래가 가능한 디지털 통'},
 {'descrip

### 메모리 관리

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini", api_key= api_key)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다."),
    ("placeholder", "{messages}"),
]
)

chain = prompt|chat

ai_msg = chain.invoke(
    {
        "messages" : [
            ("human", "저축을 늘리기 위해 무엇을 할 수 있나요"),
            ("ai", "저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요"),
            ("human", "방금 뭐라고 했나요?"),
        ],
    }
)

print(ai_msg.content)

저축을 늘리기 위해 목표를 설정하고, 매달 자동 이체를 통해 일정 금액을 저축하는 것이 좋다고 말씀드렸습니다. 이 방법은 저축을 습관화하는 데 도움이 됩니다. 추가로 다른 방법에 대해서도 설명해 드릴까요?


In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message("저축을 늘리기 위해 무엇을 할 수 있나요?")
chat_history.add_ai_message("저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")
chat_history.add_user_message("방금 뭐라고 하셨나요")

ai_response = chain.invoke({"messages" : chat_history.messages})

print(ai_response.content)

저축을 늘리기 위한 몇 가지 방법을 제안드렸습니다. 첫째, 저축 목표를 설정하여 구체적인 금액과 기간을 정하세요. 둘째, 매달 자동 이체를 설정해 일정 금액이 자동으로 저축 계좌로 들어가도록 하세요. 이렇게 하면 저축을 더 쉽게 늘릴 수 있습니다. 추가적으로 다른 방법이나 궁금한 점이 있으시면 말씀해 주세요!


In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system" , "당신은 금융 상담가입니다. 모든 질문에 최선을 다해 답변하세요."),
    ("placeholder" , "{chat_history}"),
    ("human" , "{input}"),
]
)

chat = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

chat_history = ChatMessageHistory()

chain = prompt|chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda seesion_id : chat_history,
    input_messages_key= "input",
    history_messages_key= "chat_history",
)

result1 = chain_with_message_history.invoke(
    {"input" : "저축을 늘리기 위해 무엇을 할 수 있나요?"},
    {"configurable" : {"session_id" : "unused"}},
)

print(result1.content)

result2 = chain_with_message_history.invoke(
    {"input" : "내가 무슨 질문을 했나요?"},
    {"configurable" : {"session_id" : "unused"}},
)

print(result2.content)

저축을 늘리는 방법에는 여러 가지가 있습니다. 아래 몇 가지 전략을 소개하겠습니다.

1. **예산 세우기**: 수입과 지출을 명확히 파악하여 어느 부분에서 절약할 수 있는지 분석하세요. 예산을 세우면 불필요한 지출을 줄이는 데 도움이 됩니다.

2. **자동 저축 설정**: 월급이 들어오는 즉시 일정 금액을 저축 계좌로 자동 이체하도록 설정하세요. 이렇게 하면 저축이 우선시되고, 사용 가능한 돈이 줄어들어 자연스럽게 지출을 줄일 수 있습니다.

3. **고정 비용 검토**: 정기적으로 나가는 고정 비용(예: 보험료, 통신비 등)을 검토하고, 더 저렴한 옵션으로 변경할 수 있는지를 확인하세요.

4. **비상 기금 마련**: 예상치 못한 지출을 대비해 비상 기금을 마련하세요. 이 기금은 필요할 때 빠르게 사용할 수 있도록 별도의 계좌에 보관하는 것이 좋습니다.

5. **소비 습관 점검**: 충동 구매를 줄이고, 필요 없는 물건을 사지 않도록 스스로에게 질문해 보세요. '이 물건이 나에게 정말 필요한가?'를 자주 생각해보세요.

6. **할인 및 프로모션 활용**: 구매 전에 항상 할인 코드나 프로모션을 검색하여 절약할 수 있는 기회를 활용하세요.

7. **부수입 늘리기**: 가능한 경우, 아르바이트나 프리랜스 일을 통해 추가 수입을 만드는 것도 좋은 방법입니다.

8. **저축 목표 설정**: 단기 및 장기 저축 목표를 설정하고, 그 목표를 달성하기 위한 계획을 수립하세요. 목표가 있으면 저축 동기를 부여 받을 수 있습니다.

이와 같은 방법들을 통해 저축을 체계적으로 늘려나갈 수 있습니다. 각 개인의 상황에 맞는 전략을 선택하여 실천해 보세요!
당신이 하신 질문은 "저축을 늘리기 위해 무엇을 할 수 있나요?"입니다. 이 질문에 대해 저축을 늘리는 다양한 방법과 전략을 제안드렸습니다. 추가적인 질문이나 더 구체적인 정보가 필요하시면 언제든지 말씀해 주세요!


In [24]:
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from operator import itemgetter
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system" , "당신은 금융 상담가입니다. 모든 질문에 최선을 다해 답변하세요."),
    ("placeholder" , "{chat_history}"),
    ("human" , "{input}"),
]
)

chat = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

chat_history = ChatMessageHistory()

trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

chain_with_trimming = (
    RunnablePassthrough.assign(chat_history = itemgetter("chat_history") | trimmer)
    | prompt
    | chat
)

chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda seesion_id : chat_history,
    input_messages_key="input",
    history_messages_key= "chat_history",
)

chain_with_trimmed_history.invoke(
    {"input" : "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"},
    { "configurable" : {"session_id" : "finance_session_1"}}
)

chain_with_trimmed_history.invoke(
    {"input" : "내가 방금 뭐라고 했나요"},
    {"configurable" : {"session_id" : "finance_seesion_1"}}
)




AIMessage(content='당신은 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하는지에 대해 질문하셨습니다. 이를 위해 체계적이고 세밀한 재정 계획을 세우는 방법에 대한 조언을 드렸습니다. 추가적으로 궁금한 점이 있으시면 언제든지 물어보세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 754, 'total_tokens': 827, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BnogMnZBUTSkkVktr1JDOL0n5RD1r', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d25378a3-f671-460f-a729-5598c084e26f-0', usage_metadata={'input_tokens': 754, 'output_tokens': 73, 'total_tokens': 827, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_openai import ChatOpenAI

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if( len(stored_messages) == 0 ):
        return False

    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ( "placeholder" , "{chat_history}" ),
            ("user" , "이전 대화를 요약해 주세요. 가능한 한 많은 세부 정보를 포함하십시오"),  
        ]
    )

    summarization_chain = summarization_prompt | chat
    summary_message = summarization_chain.invoke({"chat_history":stored_messages})

    chat_history.clear()
    chat_history.add_message(summary_message)
    return True

prompt = ChatPromptTemplate.from_messages([
    ("system" , "당신은 금융 상담가입니다. 모든 질문에 최선을 다해 답변하세요."),
    ("placeholder" , "{chat_history}"),
    ("human" , "{input}"),
]
)

chat = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

chat_history = ChatMessageHistory()

chain = prompt|chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda seesion_id : chat_history,
    input_messages_key= "input",
    history_messages_key= "chat_history",
)


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized = summarize_messages)
    | chain_with_message_history
)

result = chain_with_summarization.invoke(
    {"input" : "저에게 어떤 재정적 조언을 해주셨나요?"},
    {"configurable" : { "session_id" : "unused" } }
)

print(result.content)

제가 드린 재정적 조언은 주로 저축을 늘리고 재정 관리를 개선하기 위한 여러 전략이었습니다. 다음은 구체적인 조언 요약입니다:

1. **예산 세우기**: 월별 수입과 지출을 기록하여 예산을 세우고, 필요 없는 지출을 줄이는 방법을 찾으세요.

2. **자동 저축 설정**: 월급의 일정 금액을 저축 계좌로 자동 이체하여 저축이 자연스러운 습관이 되도록 하세요.

3. **고정 비용 검토**: 정기적으로 나가는 지출을 점검하고, 더 저렴한 대체 상품이나 서비스를 찾아보세요.

4. **비상 기금 마련**: 세 개월치 생활비 정도는 비상용으로 저축해두어 예기치 않은 지출에 대비하세요.

5. **소비 습관 점검**: 필요 없는 소비를 줄이고, 꼭 필요한 구매인지 생각해보세요. 

6. **할인 및 프로모션 활용**: 구매 전 항상 할인 코드를 검색하고, 세일을 활용하여 절약하세요.

7. **부수입 늘리기**: 아르바이트, 프리랜서 일 등으로 추가 수입을 얻는 방법을 고려해 보세요.

8. **저축 목표 설정**: 실현 가능한 단기 및 장기 저축 목표를 세우고, 목표를 위한 계획을 세우면서 동기를 부여하세요.

이 조언들을 통해 재정 관리를 좀 더 효과적으로 할 수 있을 것입니다. 추가적인 질문이나 더 필요한 정보가 있으면 언제든지 말씀해 주세요!
